In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
import numpy as np

In [5]:
a=pd.read_csv(r'C:\Users\49653\Desktop\yx\data\newdata.csv',encoding='utf-8')
target=np.array(a['label'])
data=np.array(a.drop(['label'],axis=1))
X=data
Y=target
X_train,X_test,y_train,y_test= train_test_split(data,target,test_size=0.1,random_state=0)

In [24]:
def AdaBoost():
    clf = AdaBoostClassifier(n_estimators=100)  ##AdaBoost弱分类器
    clf.fit(X_train,y_train)
    yad_predict=clf.predict(X_test)
    scores = cross_val_score(clf, X_train,y_train,cv=5)
    print("模型拟合度为:",scores)
    print(classification_report(y_test,yad_predict))
    plt.plot(yad_predict, label='pre')
    plt.plot(y_test, label='true')
    plt.legend()
    plt.show()
def NBG():  ##朴素贝叶斯
    clf= GaussianNB()
    clf.fit(X_train,y_train)
    ynbg_predict=clf.predict(X_test)
    plt.plot(ynbg_predict, label='pre')
    plt.plot(y_test, label='true')
    plt.legend()
    plt.show()
    scores=clf.score(X_test,y_test)
    print("模型拟合度为:",scores)
    print(classification_report(y_test,ynbg_predict)) 
def NBM():  ##朴素贝叶斯
    clf=MultinomialNB()
    clf.fit(X_train,y_train)
    ynbm_predict=clf.predict(X_test)
    plt.plot(ynbm_predict, label='pre')
    plt.plot(y_test, label='true')
    plt.legend()
    plt.show()
    scores=clf.score(X_test,y_test)
    print("模型拟合度为:",scores)
    print(classification_report(y_test,ynbm_predict))
def desic():
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    ydesic_predict=clf.predict(X_test)
    plt.plot(ydesic_predict, label='pre')
    plt.plot(y_test, label='true')
    plt.legend()
    plt.show()
    scores=clf.score(X_test,y_test)
    print("模型拟合度为:",scores)
    print(classification_report(y_test,ydesic_predict)) 

In [30]:
import streamlit as st

In [37]:
an=st.radio('AdaBoost','NBG','NBM','desic',("AdaBoost","NBG","NBM","desic"))
try:
    if an=="AdaBoost":
        st.info(AdaBoost())
    elif an=='NBG':
        st.info(NBG())
    elif an=='NBM':
        st.info(NBM())
    elif an=='desic':
        st.info(desic())
except:
    print(None)

StreamlitAPIException: Radio Value has invalid type: str

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import sys
from IPython import display
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  ##因为我的GPU在第0个，所以特此标记
torch.cuda.set_device(0)
# 读取数据：
train_data = pd.read_csv('train.csv',encoding='utf-8')
test_data = pd.read_csv('test.csv',encoding='utf-8')
print(train_data.shape)
print(test_data.shape)
# 连接训练数据与测试数据
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
print(all_features.head(5))

# 数据预处理：
# 选泽合适索引：
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
# 标准化：
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
# 缺失值补充：
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# 离散数据转换成指示特征：
# pd.get_dummies(data,prefix,columns,dummy_na)
# prefix:转换后的前缀
# columns:需要转换的列名
# dummy_na:是否增加一列表示空缺值
all_features = pd.get_dummies(all_features, dummy_na=True)
print(all_features)
# 通过values得到Numpy的数据，并转换成Tensor
n_train = train_data.shape[0]  # 样本个数
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float)  # 前n_train行
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float)  # n_train行后
train_labels = torch.tensor(train_data.label.values, dtype=torch.float).view(-1, 1)  # 标签列
# 训练模型

# 损失函数：MSE（均方差）
loss = nn.MSELoss()


# 神经网络：
def get_net(feature_num):
    net = nn.Linear(feature_num, 1)

    # 初始化模型参数：
    # parameters()返回一个迭代器，每次生成的是tensor类型的数据：
    for param in net.parameters():
        nn.init.normal_(param, mean=0, std=0.01)

    return net


# 对数均方根误差的实现：
def log_rmse(net, features, labels):
    with torch.no_grad():
        # 小于1的数设为1，取对数稳定
        clipped_preds = torch.max(net(features), torch.tensor(1.0))
        rmse = torch.sqrt(2 * loss(clipped_preds.float().log(), labels.float().log()).mean())
    return rmse.item()  # item():得到该值


# 训练数据
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []  # 损失
    dataset = utils.data.TensorDataset(train_features, train_labels)
    train_iter = utils.data.DataLoader(dataset, batch_size, shuffle=True)
    # 优化器：Adam
    optimizer = optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=weight_decay)
    net = net.float()
    for epoch in range(num_epochs):
        for x, y in train_iter:
            # 正向传播：
            output = net(x.float())
            # 计算损失：
            l = loss(output, y.float())
            # 梯度归零
            optimizer.zero_grad()
            # 反向传播
            l.backward()
            # 优化参数
            optimizer.step()

        train_ls.append(log_rmse(net, train_features, train_labels))  # 训练集
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))  # 测试集

    return train_ls, test_ls


# K折交叉验证
def get_k_fold_data(k, i, x, y):
    # 返回第i折交叉验证需要的训练（x_train）和测试数据（x_test）
    assert k > 1
    fold_size = x.shape[0] // k
    x_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)  # 切片函数
        x_part, y_part = x[idx, :], y[idx]
        if j == i:  # 恰好i==j，直接载入测试集
            x_test, y_test = x_part, y_part
        elif x_train is None:  # 载入训练集
            x_train, y_train = x_part, y_part
        else:  # 将各个训练集相加
            x_train = torch.cat((x_train, x_part), dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
    return x_train, y_train, x_test, y_test


def use_svg_display():
    # 用矢量图表示：
    display.set_matplotlib_formats('svg')


def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    # 设置图像尺寸
    plt.rcParams['figure.figsize'] = figsize


def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None, y2_vals=None,
             legend=None, figsize=(3.5, 2.5)):
    set_figsize(figsize)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.semilogy(x_vals, y_vals)  # semilogy:半对数函数
    if x2_vals and y2_vals:
        plt.semilogy(x2_vals, y2_vals, linestyle=':')
        plt.legend(legend)

    plt.show()


def k_fold(k, x_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, x_train, y_train)
        net = get_net(x_train.shape[1])
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse',
                     range(1, num_epochs + 1), valid_ls, ['train', 'valid'])
        print('fold %d, train rmse %f, valid rmse %f' % (i, train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k  # 平均损失


k, num_epochs, lr, weight_decay, batch_size = 10, 100, 5, 1.5, 256
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
print('%d-fold validation: avg train rmse %f,avg valid rmse %f' % (k, train_l, valid_l))


def train_and_pred(train_features, test_features, train_labels, test_data,
                   num_epochs, learning_rate, weight_decay, batch_size):
    net = get_net(train_features.shape[1])
    train_ls, _ = train(net, train_features, train_labels, None, None, num_epochs,
                        learning_rate, weight_decay, batch_size)
    semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse')
    print('train rmse %f' % train_ls[-1])
    preds = net(test_features).detach().numpy()  ##神经网络对其进行预测

    test_data['label'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['length'], test_data['label']], axis=1)
    submission.to_csv('submission.csv', index=False)


train_and_pred(train_features, test_features, train_labels, test_data,
               num_epochs, lr, weight_decay, batch_size)
